In [1]:
import os
import geopandas as gpd
import gdal
import numpy as np
import numba
import glob
from multiprocessing import Pool
import tqdm.auto as tqdm

In [8]:
%%time
geotiff_folder = r'/media/nick/2TB Working 2/Projects/GeoTIFF_tiler_data/VIVID_Landgate_20190910_112102323 (copy)'
geo_tiff_list = glob.glob(geotiff_folder+'/*.tif')
print(len(geo_tiff_list),'files')

272 files
CPU times: user 1.9 ms, sys: 306 µs, total: 2.2 ms
Wall time: 1.52 ms


In [9]:
def get_bounds(tif_path):
#     open file
    data = gdal.Open(tif_path)
#     grab bounds
    geoTransform = data.GetGeoTransform()
    left = geoTransform[0]
    top = geoTransform[3]
    right = left + geoTransform[1] * data.RasterXSize
    bottom = top + geoTransform[5] * data.RasterYSize
    return[top, left, bottom, right]

In [11]:
%%time
# use multiprocessing to extract raster bounds
with Pool() as pool:
    geo_tiff_bounds = list(tqdm.tqdm(pool.imap(get_bounds, geo_tiff_list), total=len(geo_tiff_list)))
# convert list to np array
geo_tiff_bounds = np.array(geo_tiff_bounds)
# grab max extents
top_max = geo_tiff_bounds[:,0].max()
left_max = geo_tiff_bounds[:,1].min()
bottom_max = geo_tiff_bounds[:,2].min()
right_max = geo_tiff_bounds[:,3].max()

  0%|          | 0/272 [00:00<?, ?it/s]

CPU times: user 37 ms, sys: 32.4 ms, total: 69.4 ms
Wall time: 143 ms
